In [2]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
print("mean length = ",(len([x for item in all_training_text for x in item]))/len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")



txt_files of features used ['/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_chosen_set.csv', '/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_chosen_set.csv']
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_chosen_set.csv 5531
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_chosen_set.csv 7464
length of features used = 12995
len(all_training_text) 383
mean length =  9713.52219321149
len(X_train)= 383 len(y_train)= 383
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
manhattan K-Nearest Neighbours
0.5512820512820513
cosine K-Nearest Neighbours
0.5384615384615384
euclidean K-Nearest Neighbours
0.5384615384615384
braycurtis K-Nearest Neighbours
0.6282051282051282
canberra K-Nearest Neighbours
0.5
jaccard K-Nearest Neighbours
0.6666666666666666


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


yule K-Nearest Neighbours
0.6282051282051282
************************************ ****************************


In [9]:
"""
0.5512820512820513
0.5384615384615384
0.5384615384615384
0.6282051282051282
0.5
0.6666666666666666
0.6282051282051282

  Manhattan 0.5512820512820513
  Canberra  0.5
  Bray-Curtis 0.628
  Cosine  0.539
  Jaccard 0.667
  yule. 0.628
"""

import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.2F.txt"]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        #reader = csv.reader(f, delimiter=",")
        #next(reader) # skip header
        word_list1 =  ([(r.split(":"))[1] for r in f])[0:100]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        reader = csv.reader(f, delimiter=",")
#        next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")





txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.1M.txt', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.2F.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.1M.txt 100
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.2F.txt 100
length of features used = 200
len(all_training_text) 383
len(X_train)= 383 len(y_train)= 383
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
manhattan K-Nearest Neighbours
0.5256410256410257
cosine K-Nearest Neighbours
0.5
euclidean K-Nearest Neighbours
0.5256410256410257
braycurtis K-Nearest Neighbours
0.5256410256410257
canberra K-Nearest Neighbours
0.5256410256410257
jaccard K-Nearest Neighbours
0.5
yule K-Nearest Neighbours
0.5
************************************ ****************************


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [10]:
"""
  Manhattan 0.53
  Canberra  0.53
  Bray-Curtis 0.53
  Cosine  0.50
  Jaccard 0.50
  yule. 0.50
"""

import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.2F.txt"]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        #reader = csv.reader(f, delimiter=",")
        #next(reader) # skip header
        word_list1 =  ([(r.split(":"))[1] for r in f])[0:150]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        reader = csv.reader(f, delimiter=",")
#        next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")






txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.1M.txt', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.2F.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.1M.txt 150
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.2F.txt 150
length of features used = 300
len(all_training_text) 383
len(X_train)= 383 len(y_train)= 383
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
manhattan K-Nearest Neighbours
0.5256410256410257
cosine K-Nearest Neighbours
0.5
euclidean K-Nearest Neighbours
0.5256410256410257
braycurtis K-Nearest Neighbours
0.5256410256410257
canberra K-Nearest Neighbours
0.5256410256410257
jaccard K-Nearest Neighbours
0.5128205128205128
yule K-Nearest Neighbours
0.5
************************************ ****************************


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [11]:
"""

  Manhattan 0.53
  Canberra  0.53
  Bray-Curtis 0.53
  Cosine  0.50
  Jaccard 0.51
  yule. 0.50
"""

import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.2F.txt"]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        #reader = csv.reader(f, delimiter=",")
        #next(reader) # skip header
        word_list1 =  ([(r.split(":"))[1] for r in f])[0:250]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        reader = csv.reader(f, delimiter=",")
#        next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")






txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.1M.txt', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.2F.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.1M.txt 250
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.2F.txt 250
length of features used = 500
len(all_training_text) 383
len(X_train)= 383 len(y_train)= 383
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
manhattan K-Nearest Neighbours
0.5256410256410257
cosine K-Nearest Neighbours
0.5
euclidean K-Nearest Neighbours
0.5256410256410257
braycurtis K-Nearest Neighbours
0.5256410256410257
canberra K-Nearest Neighbours
0.5256410256410257
jaccard K-Nearest Neighbours
0.5128205128205128
yule K-Nearest Neighbours
0.5
************************************ ****************************


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [12]:
"""
  Manhattan 0.53
  Canberra  0.53
  Bray-Curtis 0.53
  Cosine  0.50
  Jaccard 0.51
  yule. 0.628
"""

import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016PMI.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016PMI.2F.txt"]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        #reader = csv.reader(f, delimiter=",")
        #next(reader) # skip header
        word_list1 =  ([(r.split(":"))[1] for r in f])[0:100]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        reader = csv.reader(f, delimiter=",")
#        next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")






txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016PMI.1M.txt', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016PMI.2F.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016PMI.1M.txt 100
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016PMI.2F.txt 100
length of features used = 200
len(all_training_text) 383
len(X_train)= 383 len(y_train)= 383
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
manhattan K-Nearest Neighbours
0.5256410256410257
cosine K-Nearest Neighbours
0.5
euclidean K-Nearest Neighbours
0.5256410256410257
braycurtis K-Nearest Neighbours
0.5256410256410257
canberra K-Nearest Neighbours
0.5256410256410257
jaccard K-Nearest Neighbours
0.5
yule K-Nearest Neighbours
0.5
************************************ ****************************


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [13]:
"""
  Manhattan 0.53
  Canberra  0.53
  Bray-Curtis 0.53
  Cosine  0.50
  Jaccard 0.50
  yule. 0.50
"""

import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016PMI.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016PMI.2F.txt"]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        #reader = csv.reader(f, delimiter=",")
        #next(reader) # skip header
        word_list1 =  ([(r.split(":"))[1] for r in f])[0:150]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        reader = csv.reader(f, delimiter=",")
#        next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")







txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016PMI.1M.txt', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016PMI.2F.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016PMI.1M.txt 150
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016PMI.2F.txt 150
length of features used = 300
len(all_training_text) 383
len(X_train)= 383 len(y_train)= 383
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
manhattan K-Nearest Neighbours
0.5256410256410257
cosine K-Nearest Neighbours
0.5
euclidean K-Nearest Neighbours
0.5256410256410257
braycurtis K-Nearest Neighbours
0.5256410256410257
canberra K-Nearest Neighbours
0.5256410256410257
jaccard K-Nearest Neighbours
0.5128205128205128
yule K-Nearest Neighbours
0.5
************************************ ****************************


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [14]:
"""
0.5512820512820513
0.5384615384615384
0.5384615384615384
0.6282051282051282
0.5
0.6666666666666666
0.6282051282051282

  Manhattan 0.5512820512820513
  Canberra  0.5
  Bray-Curtis 0.628
  Cosine  0.539
  Jaccard 0.667
  yule. 0.628
"""

import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016PMI.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016PMI.2F.txt"]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        #reader = csv.reader(f, delimiter=",")
        #next(reader) # skip header
        word_list1 =  ([(r.split(":"))[1] for r in f])[0:250]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        reader = csv.reader(f, delimiter=",")
#        next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")







txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016PMI.1M.txt', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016PMI.2F.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016PMI.1M.txt 250
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016PMI.2F.txt 250
length of features used = 500
len(all_training_text) 383
len(X_train)= 383 len(y_train)= 383
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
manhattan K-Nearest Neighbours
0.5256410256410257
cosine K-Nearest Neighbours
0.5
euclidean K-Nearest Neighbours
0.5256410256410257
braycurtis K-Nearest Neighbours
0.5256410256410257
canberra K-Nearest Neighbours
0.5256410256410257
jaccard K-Nearest Neighbours
0.5128205128205128
yule K-Nearest Neighbours
0.5
************************************ ****************************


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [16]:
#ProbD
"""
0.5512820512820513
0.5384615384615384
0.5384615384615384
0.6282051282051282
0.5
0.6666666666666666
0.6282051282051282

  Manhattan 0.5512820512820513
  Canberra  0.5
  Bray-Curtis 0.628
  Cosine  0.539
  Jaccard 0.667
  yule. 0.628
"""

import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016ProbD.2F.txt"]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        #reader = csv.reader(f, delimiter=",")
        #next(reader) # skip header
        word_list1 =  ([(r.split(":"))[1] for r in f])[0:100]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        reader = csv.reader(f, delimiter=",")
#        next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")








txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016ProbD.1M.txt', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016ProbD.2F.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016ProbD.1M.txt 100
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016ProbD.2F.txt 100
length of features used = 200
len(all_training_text) 383
len(X_train)= 383 len(y_train)= 383
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
manhattan K-Nearest Neighbours
0.5
cosine K-Nearest Neighbours
0.5
euclidean K-Nearest Neighbours
0.5
braycurtis K-Nearest Neighbours
0.5
canberra K-Nearest Neighbours
0.5
jaccard K-Nearest Neighbours
0.5
yule K-Nearest Neighbours
0.5
************************************ ****************************


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [18]:
#ProbD
"""
0.5512820512820513
0.5384615384615384
0.5384615384615384
0.6282051282051282
0.5
0.6666666666666666
0.6282051282051282

  Manhattan 0.5512820512820513
  Canberra  0.5
  Bray-Curtis 0.628
  Cosine  0.539
  Jaccard 0.667
  yule. 0.628
"""

import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016CHI.txt"]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        #reader = csv.reader(f, delimiter=",")
        #next(reader) # skip header
        word_list =  ([(r.split(":"))[1] for r in f])[0:200]
        print(txt_file, len(word_list1))
        #word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        reader = csv.reader(f, delimiter=",")
#        next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")









txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016CHI.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016CHI.txt 100
length of features used = 200
len(all_training_text) 383
len(X_train)= 383 len(y_train)= 383
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
manhattan K-Nearest Neighbours
0.5256410256410257
cosine K-Nearest Neighbours
0.5
euclidean K-Nearest Neighbours
0.5256410256410257
braycurtis K-Nearest Neighbours
0.5256410256410257
canberra K-Nearest Neighbours
0.5256410256410257
jaccard K-Nearest Neighbours
0.5
yule K-Nearest Neighbours
0.5
************************************ ****************************


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [19]:
#ProbD
"""

  Manhattan 0.53
  Canberra  0.53
  Bray-Curtis 0.53
  Cosine  0.50
  Jaccard 0.50
  yule. 0.50
"""

import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016CHI.txt"]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        #reader = csv.reader(f, delimiter=",")
        #next(reader) # skip header
        word_list =  ([(r.split(":"))[1] for r in f])[0:300]
        print(txt_file, len(word_list1))
        #word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        reader = csv.reader(f, delimiter=",")
#        next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")










txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016CHI.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016CHI.txt 100
length of features used = 300
len(all_training_text) 383
len(X_train)= 383 len(y_train)= 383
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
manhattan K-Nearest Neighbours
0.5256410256410257
cosine K-Nearest Neighbours
0.5
euclidean K-Nearest Neighbours
0.5256410256410257
braycurtis K-Nearest Neighbours
0.5256410256410257
canberra K-Nearest Neighbours
0.5256410256410257
jaccard K-Nearest Neighbours
0.5
yule K-Nearest Neighbours
0.5
************************************ ****************************


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [20]:
#ProbD
"""

  Manhattan 0.53
  Canberra  0.53
  Bray-Curtis 0.53
  Cosine  0.50
  Jaccard 0.50
  yule. 0.50
"""

import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016CHI.txt"]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        #reader = csv.reader(f, delimiter=",")
        #next(reader) # skip header
        word_list =  ([(r.split(":"))[1] for r in f])[0:500]
        print(txt_file, len(word_list1))
        #word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        reader = csv.reader(f, delimiter=",")
#        next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")











txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016CHI.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016CHI.txt 100
length of features used = 500
len(all_training_text) 383
len(X_train)= 383 len(y_train)= 383
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
manhattan K-Nearest Neighbours
0.5256410256410257
cosine K-Nearest Neighbours
0.5
euclidean K-Nearest Neighbours
0.5256410256410257
braycurtis K-Nearest Neighbours
0.5256410256410257
canberra K-Nearest Neighbours
0.5256410256410257
jaccard K-Nearest Neighbours
0.5
yule K-Nearest Neighbours
0.5
************************************ ****************************


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [21]:

#ProbD
"""

  Manhattan 0.53
  Canberra  0.53
  Bray-Curtis 0.53
  Cosine  0.50
  Jaccard 0.50
  yule. 0.50
"""

import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016GSS.txt"]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        #reader = csv.reader(f, delimiter=",")
        #next(reader) # skip header
        word_list =  ([(r.split(":"))[1] for r in f])[0:200]
        print(txt_file, len(word_list1))
        #word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        reader = csv.reader(f, delimiter=",")
#        next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")












txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016GSS.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016GSS.txt 100
length of features used = 200
len(all_training_text) 383
len(X_train)= 383 len(y_train)= 383
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
manhattan K-Nearest Neighbours
0.5
cosine K-Nearest Neighbours
0.5
euclidean K-Nearest Neighbours
0.5
braycurtis K-Nearest Neighbours
0.5
canberra K-Nearest Neighbours
0.5
jaccard K-Nearest Neighbours
0.5
yule K-Nearest Neighbours
0.5
************************************ ****************************


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [22]:

#ProbD
"""

  Manhattan 0.53
  Canberra  0.53
  Bray-Curtis 0.53
  Cosine  0.50
  Jaccard 0.50
  yule. 0.50
"""

import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016GSS.txt"]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        #reader = csv.reader(f, delimiter=",")
        #next(reader) # skip header
        word_list =  ([(r.split(":"))[1] for r in f])[0:300]
        print(txt_file, len(word_list1))
        #word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        reader = csv.reader(f, delimiter=",")
#        next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")













txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016GSS.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016GSS.txt 100
length of features used = 300
len(all_training_text) 383
len(X_train)= 383 len(y_train)= 383
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
manhattan K-Nearest Neighbours
0.5
cosine K-Nearest Neighbours
0.5
euclidean K-Nearest Neighbours
0.5
braycurtis K-Nearest Neighbours
0.5
canberra K-Nearest Neighbours
0.5
jaccard K-Nearest Neighbours
0.5
yule K-Nearest Neighbours
0.5
************************************ ****************************


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [3]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
print("mean length = ",(len([x for item in all_training_text for x in item]))/len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")


txt_files of features used ['/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv 6328
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv 8793
length of features used = 15121
len(all_training_text) 383
mean length =  14816.32637075718
len(X_train)= 383 len(y_train)= 383
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
manhattan K-Nearest Neighbours
0.5128205128205128
cosine K-Nearest Neighbours
0.5384615384615384
euclidean K-Nearest Neighbours
0.5128205128205128
braycurtis K-Nearest Neighbours
0.5256410256410257
canberra K-Nearest Neighbours
0.5
jaccard K-Nearest Neighbours
0.6923076923076923


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


yule K-Nearest Neighbours
0.6282051282051282
************************************ ****************************


In [24]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "//Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=chi2, k=200)")
selector = SelectKBest(score_func=chi2, k=200)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")


txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv 6328
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv 8793
length of features used = 15121
len(all_training_text) 383
len(X_train)= 383 len(y_train)= 383
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
selector = SelectKBest(score_func=chi2, k=200)
5
manhattan K-Nearest Neighbours
0.5256410256410257
cosine K-Nearest Neighbours
0.48717948717948717
euclidean K-Nearest Neighbours
0.48717948717948717
braycurtis K-Nearest Neighbours
0.5256410256410257
canberra K-Nearest Neighbours
0.48717948717948717
jaccard K-Nearest Neighbours
0.5128205128205128
yule K-Nearest Neighbours
0.5641025641025641


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [25]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "//Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=chi2, k=300)")
selector = SelectKBest(score_func=chi2, k=300)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")



txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv 6328
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv 8793
length of features used = 15121
len(all_training_text) 383
len(X_train)= 383 len(y_train)= 383
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
selector = SelectKBest(score_func=chi2, k=300)
5
manhattan K-Nearest Neighbours
0.5256410256410257
cosine K-Nearest Neighbours
0.5256410256410257
euclidean K-Nearest Neighbours
0.5128205128205128
braycurtis K-Nearest Neighbours
0.5384615384615384
canberra K-Nearest Neighbours
0.46153846153846156
jaccard K-Nearest Neighbours
0.5769230769230769
yule K-Nearest Neighbours
0.6025641025641025
**

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [26]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "//Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=chi2, k=500)")
selector = SelectKBest(score_func=chi2, k=500)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")




txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv 6328
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv 8793
length of features used = 15121
len(all_training_text) 383
len(X_train)= 383 len(y_train)= 383
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
selector = SelectKBest(score_func=chi2, k=500)
5
manhattan K-Nearest Neighbours
0.5256410256410257
cosine K-Nearest Neighbours
0.5256410256410257
euclidean K-Nearest Neighbours
0.5128205128205128
braycurtis K-Nearest Neighbours
0.5512820512820513
canberra K-Nearest Neighbours
0.48717948717948717
jaccard K-Nearest Neighbours
0.6282051282051282
yule K-Nearest Neighbours
0.6794871794871795
**

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [27]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "//Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=mutual_info_classif, k=200)")
selector = SelectKBest(score_func=mutual_info_classif, k=200)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")




txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv 6328
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv 8793
length of features used = 15121
len(all_training_text) 383
len(X_train)= 383 len(y_train)= 383
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
selector = SelectKBest(score_func=mutual_info_classif, k=200)
5
manhattan K-Nearest Neighbours
0.5897435897435898
cosine K-Nearest Neighbours
0.5641025641025641
euclidean K-Nearest Neighbours
0.5897435897435898
braycurtis K-Nearest Neighbours
0.5512820512820513
canberra K-Nearest Neighbours
0.48717948717948717
jaccard K-Nearest Neighbours
0.5384615384615384
yule K-Nearest Neighbours
0.5256

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [28]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "//Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=mutual_info_classif, k=300)")
selector = SelectKBest(score_func=mutual_info_classif, k=300)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")





txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv 6328
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv 8793
length of features used = 15121
len(all_training_text) 383
len(X_train)= 383 len(y_train)= 383
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
selector = SelectKBest(score_func=mutual_info_classif, k=300)
5
manhattan K-Nearest Neighbours
0.5
cosine K-Nearest Neighbours
0.47435897435897434
euclidean K-Nearest Neighbours
0.5
braycurtis K-Nearest Neighbours
0.44871794871794873
canberra K-Nearest Neighbours
0.5256410256410257
jaccard K-Nearest Neighbours
0.5256410256410257
yule K-Nearest Neighbours
0.6153846153846154
****************

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [29]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "//Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=mutual_info_classif, k=500)")
selector = SelectKBest(score_func=mutual_info_classif, k=500)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")


txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv 6328
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv 8793
length of features used = 15121
len(all_training_text) 383
len(X_train)= 383 len(y_train)= 383
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
selector = SelectKBest(score_func=mutual_info_classif, k=500)
5
manhattan K-Nearest Neighbours
0.5
cosine K-Nearest Neighbours
0.48717948717948717
euclidean K-Nearest Neighbours
0.5256410256410257
braycurtis K-Nearest Neighbours
0.48717948717948717
canberra K-Nearest Neighbours
0.5
jaccard K-Nearest Neighbours
0.6025641025641025
yule K-Nearest Neighbours
0.6025641025641025
****************

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [31]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "//Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_vocubulary.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_ vocubulary_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
#print("selector = SelectKBest(score_func=chi2, k=200)")
#selector = SelectKBest(score_func=chi2, k=200)
#fit = selector.fit(X_train,y_train)
#train_selected_features = fit.transform(X_train)
#test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
#X_train =train_selected_features 
#X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")



txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_vocubulary.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_ vocubulary_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_vocubulary.1.csv 115184
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_ vocubulary_set.1.csv 139886
length of features used = 255070
len(all_training_text) 383
len(X_train)= 383 len(y_train)= 383
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
manhattan K-Nearest Neighbours
0.5256410256410257
cosine K-Nearest Neighbours
0.5384615384615384
euclidean K-Nearest Neighbours
0.5128205128205128
braycurtis K-Nearest Neighbours
0.5384615384615384
canberra K-Nearest Neighbours
0.5384615384615384
jaccard K-Nearest Neighbours
0.6538461538461539


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


yule K-Nearest Neighbours
0.6410256410256411
************************************ ****************************


In [32]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "//Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=f_classif, k=200)")
selector = SelectKBest(score_func=f_classif, k=200)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")



txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv 6328
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv 8793
length of features used = 15121
len(all_training_text) 383
len(X_train)= 383 len(y_train)= 383
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
selector = SelectKBest(score_func=f_classif, k=200)
5
manhattan K-Nearest Neighbours
0.6025641025641025
cosine K-Nearest Neighbours
0.5512820512820513
euclidean K-Nearest Neighbours
0.5
braycurtis K-Nearest Neighbours
0.5256410256410257
canberra K-Nearest Neighbours
0.5128205128205128
jaccard K-Nearest Neighbours
0.6538461538461539
yule K-Nearest Neighbours
0.5641025641025641
*************

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [33]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "//Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=f_classif, k=300)")
selector = SelectKBest(score_func=f_classif, k=300)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")




txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv 6328
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv 8793
length of features used = 15121
len(all_training_text) 383
len(X_train)= 383 len(y_train)= 383
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
selector = SelectKBest(score_func=f_classif, k=300)
5
manhattan K-Nearest Neighbours
0.5897435897435898
cosine K-Nearest Neighbours
0.5384615384615384
euclidean K-Nearest Neighbours
0.5128205128205128
braycurtis K-Nearest Neighbours
0.5512820512820513
canberra K-Nearest Neighbours
0.5512820512820513
jaccard K-Nearest Neighbours
0.6153846153846154
yule K-Nearest Neighbours
0.589743589743589

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [34]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "//Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=f_classif, k=500)")
selector = SelectKBest(score_func=f_classif, k=500)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")




txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv 6328
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv 8793
length of features used = 15121
len(all_training_text) 383
len(X_train)= 383 len(y_train)= 383
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
selector = SelectKBest(score_func=f_classif, k=500)
5
manhattan K-Nearest Neighbours
0.5256410256410257
cosine K-Nearest Neighbours
0.48717948717948717
euclidean K-Nearest Neighbours
0.47435897435897434
braycurtis K-Nearest Neighbours
0.5128205128205128
canberra K-Nearest Neighbours
0.5641025641025641
jaccard K-Nearest Neighbours
0.6153846153846154
yule K-Nearest Neighbours
0.6025641025641

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [35]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "//Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_vocubulary.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_ vocubulary_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
#print("selector = SelectKBest(score_func=chi2, k=200)")
#selector = SelectKBest(score_func=chi2, k=200)
#fit = selector.fit(X_train,y_train)
#train_selected_features = fit.transform(X_train)
#test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
#X_train =train_selected_features 
#X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


knn = KNeighborsClassifier(n_neighbors = k, metric= "rogerstanimoto")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("rogerstanimoto K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")


txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_vocubulary.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_ vocubulary_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_vocubulary.1.csv 115184
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_ vocubulary_set.1.csv 139886
length of features used = 255070
len(all_training_text) 383
len(X_train)= 383 len(y_train)= 383
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
manhattan K-Nearest Neighbours
0.5256410256410257
cosine K-Nearest Neighbours
0.5384615384615384
euclidean K-Nearest Neighbours
0.5128205128205128
braycurtis K-Nearest Neighbours
0.5384615384615384
canberra K-Nearest Neighbours
0.5384615384615384
jaccard K-Nearest Neighbours
0.6538461538461539


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


yule K-Nearest Neighbours
0.6410256410256411
rogerstanimoto K-Nearest Neighbours
0.47435897435897434
************************************ ****************************


In [36]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "//Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))



knn = KNeighborsClassifier(n_neighbors = k, metric= "rogerstanimoto")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("rogerstanimoto K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")




txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv 6328
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv 8793
length of features used = 15121
len(all_training_text) 383
len(X_train)= 383 len(y_train)= 383
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
manhattan K-Nearest Neighbours
0.5128205128205128
cosine K-Nearest Neighbours
0.5384615384615384
euclidean K-Nearest Neighbours
0.5128205128205128
braycurtis K-Nearest Neighbours
0.5256410256410257
canberra K-Nearest Neighbours
0.5
jaccard K-Nearest Neighbours
0.6923076923076923


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


yule K-Nearest Neighbours
0.6282051282051282
rogerstanimoto K-Nearest Neighbours
0.5256410256410257
************************************ ****************************


In [37]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "//Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

def tanimoto_distance(x,y):
    """Calculates the tanimoto (normalised manhattan) distance between the vectors x and y
            Keyword arguments:
                x,y -- the vectors between which the distance is to be calculated
    """
    try:
        iter(x)
    except TypeError:
        logging.warning( 'Argument x is not iterable. None is returned')
        return None
    try:
        iter(y)
    except TypeError:
        logging.warning( 'Argument y is not iterable. None is returned')
        return None
    numerator = sum(abs(a - b) for a, b in zip(x, y))
    denominator = sum(max(a,b) for a,b in zip(x,y))
    return numerator/denominator
"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)

"""
8.0. tanimoto_distance K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= tanimoto_distance)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("tanimoto_distance K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))





txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv 6328
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv 8793
length of features used = 15121
len(all_training_text) 383
len(X_train)= 383 len(y_train)= 383
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
tanimoto_distance K-Nearest Neighbours
0.5256410256410257


In [38]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "//Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_vocubulary.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_ vocubulary_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
#print("selector = SelectKBest(score_func=chi2, k=200)")
#selector = SelectKBest(score_func=chi2, k=200)
#fit = selector.fit(X_train,y_train)
#train_selected_features = fit.transform(X_train)
#test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
#X_train =train_selected_features 
#X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

def tanimoto_distance(x,y):
    """Calculates the tanimoto (normalised manhattan) distance between the vectors x and y
            Keyword arguments:
                x,y -- the vectors between which the distance is to be calculated
    """
    try:
        iter(x)
    except TypeError:
        logging.warning( 'Argument x is not iterable. None is returned')
        return None
    try:
        iter(y)
    except TypeError:
        logging.warning( 'Argument y is not iterable. None is returned')
        return None
    numerator = sum(abs(a - b) for a, b in zip(x, y))
    denominator = sum(max(a,b) for a,b in zip(x,y))
    return numerator/denominator
"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)

"""
8.0. tanimoto_distance K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= tanimoto_distance)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("tanimoto_distance K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))



txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_vocubulary.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_ vocubulary_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_vocubulary.1.csv 115184
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_ vocubulary_set.1.csv 139886
length of features used = 255070
len(all_training_text) 383
len(X_train)= 383 len(y_train)= 383
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
tanimoto_distance K-Nearest Neighbours
0.5384615384615384


In [39]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "//Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=f_classif, k=200)")
selector = SelectKBest(score_func=f_classif, k=200)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

def tanimoto_distance(x,y):
    """Calculates the tanimoto (normalised manhattan) distance between the vectors x and y
            Keyword arguments:
                x,y -- the vectors between which the distance is to be calculated
    """
    try:
        iter(x)
    except TypeError:
        logging.warning( 'Argument x is not iterable. None is returned')
        return None
    try:
        iter(y)
    except TypeError:
        logging.warning( 'Argument y is not iterable. None is returned')
        return None
    numerator = sum(abs(a - b) for a, b in zip(x, y))
    denominator = sum(max(a,b) for a,b in zip(x,y))
    return numerator/denominator
"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)

"""
8.0. tanimoto_distance K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= tanimoto_distance)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("tanimoto_distance K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv 6328
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv 8793
length of features used = 15121
len(all_training_text) 383
len(X_train)= 383 len(y_train)= 383
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
selector = SelectKBest(score_func=f_classif, k=200)
5
tanimoto_distance K-Nearest Neighbours
0.5256410256410257


In [40]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "//Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016CHI.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016IG.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016GSS.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016OR.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016PMI.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016PMI.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016RF.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016RF.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016WLLR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016WLLR.2F.txt"]

print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        word_list2 =  ([r.strip() for r in f])[0:200]
        word_list1 =  ([(r.split(":"))[1] for r in word_list2])
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
#print("selector = SelectKBest(score_func=f_classif, k=200)")
#selector = SelectKBest(score_func=f_classif, k=200)
#fit = selector.fit(X_train,y_train)
#train_selected_features = fit.transform(X_train)
#test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
#X_train =train_selected_features 
#X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

def tanimoto_distance(x,y):
    """Calculates the tanimoto (normalised manhattan) distance between the vectors x and y
            Keyword arguments:
                x,y -- the vectors between which the distance is to be calculated
    """
    try:
        iter(x)
    except TypeError:
        logging.warning( 'Argument x is not iterable. None is returned')
        return None
    try:
        iter(y)
    except TypeError:
        logging.warning( 'Argument y is not iterable. None is returned')
        return None
    numerator = sum(abs(a - b) for a, b in zip(x, y))
    denominator = sum(max(a,b) for a,b in zip(x,y))
    return numerator/denominator
"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)

"""
8.0. tanimoto_distance K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= tanimoto_distance)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("tanimoto_distance K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))



txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016CHI.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016CHI.txt 200
length of features used = 200
len(all_training_text) 383
len(X_train)= 383 len(y_train)= 383
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
tanimoto_distance K-Nearest Neighbours
0.5128205128205128


In [41]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "//Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016CHI.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016IG.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016GSS.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016OR.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016PMI.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016PMI.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016RF.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016RF.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016WLLR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016WLLR.2F.txt"]

print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        word_list2 =  ([r.strip() for r in f])[0:200]
        word_list1 =  ([(r.split(":"))[1] for r in word_list2])
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
#print("selector = SelectKBest(score_func=f_classif, k=200)")
#selector = SelectKBest(score_func=f_classif, k=200)
#fit = selector.fit(X_train,y_train)
#train_selected_features = fit.transform(X_train)
#test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
#X_train =train_selected_features 
#X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

def tanimoto_distance(x,y):
    """Calculates the tanimoto (normalised manhattan) distance between the vectors x and y
            Keyword arguments:
                x,y -- the vectors between which the distance is to be calculated
    """
    try:
        iter(x)
    except TypeError:
        logging.warning( 'Argument x is not iterable. None is returned')
        return None
    try:
        iter(y)
    except TypeError:
        logging.warning( 'Argument y is not iterable. None is returned')
        return None
    numerator = sum(abs(a - b) for a, b in zip(x, y))
    denominator = sum(max(a,b) for a,b in zip(x,y))
    return numerator/denominator
"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)

"""
8.0. tanimoto_distance K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= tanimoto_distance)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("tanimoto_distance K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))




txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016IG.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016IG.txt 200
length of features used = 200
len(all_training_text) 383
len(X_train)= 383 len(y_train)= 383
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
tanimoto_distance K-Nearest Neighbours
0.48717948717948717


In [42]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "//Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016CHI.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016IG.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016GSS.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016OR.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016PMI.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016PMI.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016RF.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016RF.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016WLLR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016WLLR.2F.txt"]

print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        word_list2 =  ([r.strip() for r in f])[0:200]
        word_list1 =  ([(r.split(":"))[1] for r in word_list2])
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
#print("selector = SelectKBest(score_func=f_classif, k=200)")
#selector = SelectKBest(score_func=f_classif, k=200)
#fit = selector.fit(X_train,y_train)
#train_selected_features = fit.transform(X_train)
#test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
#X_train =train_selected_features 
#X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

def tanimoto_distance(x,y):
    """Calculates the tanimoto (normalised manhattan) distance between the vectors x and y
            Keyword arguments:
                x,y -- the vectors between which the distance is to be calculated
    """
    try:
        iter(x)
    except TypeError:
        logging.warning( 'Argument x is not iterable. None is returned')
        return None
    try:
        iter(y)
    except TypeError:
        logging.warning( 'Argument y is not iterable. None is returned')
        return None
    numerator = sum(abs(a - b) for a, b in zip(x, y))
    denominator = sum(max(a,b) for a,b in zip(x,y))
    return numerator/denominator
"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)

"""
8.0. tanimoto_distance K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= tanimoto_distance)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("tanimoto_distance K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))




txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016GSS.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016GSS.txt 200
length of features used = 200
len(all_training_text) 383
len(X_train)= 383 len(y_train)= 383
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
tanimoto_distance K-Nearest Neighbours
0.44871794871794873


In [43]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "//Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016CHI.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016IG.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016GSS.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016OR.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016PMI.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016PMI.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016RF.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016RF.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016WLLR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016WLLR.2F.txt"]

print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        word_list2 =  ([r.strip() for r in f])[0:100]
        word_list1 =  ([(r.split(":"))[1] for r in word_list2])
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
#print("selector = SelectKBest(score_func=f_classif, k=200)")
#selector = SelectKBest(score_func=f_classif, k=200)
#fit = selector.fit(X_train,y_train)
#train_selected_features = fit.transform(X_train)
#test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
#X_train =train_selected_features 
#X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

def tanimoto_distance(x,y):
    """Calculates the tanimoto (normalised manhattan) distance between the vectors x and y
            Keyword arguments:
                x,y -- the vectors between which the distance is to be calculated
    """
    try:
        iter(x)
    except TypeError:
        logging.warning( 'Argument x is not iterable. None is returned')
        return None
    try:
        iter(y)
    except TypeError:
        logging.warning( 'Argument y is not iterable. None is returned')
        return None
    numerator = sum(abs(a - b) for a, b in zip(x, y))
    denominator = sum(max(a,b) for a,b in zip(x,y))
    return numerator/denominator
"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)

"""
8.0. tanimoto_distance K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= tanimoto_distance)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("tanimoto_distance K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))




txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016OR.1M.txt', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016OR.2F.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016OR.1M.txt 100
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016OR.2F.txt 100
length of features used = 200
len(all_training_text) 383
len(X_train)= 383 len(y_train)= 383
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:259: RuntimeWarning: invalid value encountered in double_scalars


tanimoto_distance K-Nearest Neighbours
0.5384615384615384


In [47]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "//Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016CHI.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016IG.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016GSS.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016PMI.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016PMI.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016RF.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016RF.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016WLLR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016WLLR.2F.txt"]

print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        word_list2 =  ([r.strip() for r in f])[0:100]
        word_list1 =  ([(r.split(":"))[1] for r in word_list2])
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
#print("selector = SelectKBest(score_func=f_classif, k=200)")
#selector = SelectKBest(score_func=f_classif, k=200)
#fit = selector.fit(X_train,y_train)
#train_selected_features = fit.transform(X_train)
#test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
#X_train =train_selected_features 
#X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

def tanimoto_distance(x,y):
    """Calculates the tanimoto (normalised manhattan) distance between the vectors x and y
            Keyword arguments:
                x,y -- the vectors between which the distance is to be calculated
    """
    try:
        iter(x)
    except TypeError:
        logging.warning( 'Argument x is not iterable. None is returned')
        return None
    try:
        iter(y)
    except TypeError:
        logging.warning( 'Argument y is not iterable. None is returned')
        return None
    numerator = sum(abs(a - b) for a, b in zip(x, y))
    denominator = sum(max(a,b) for a,b in zip(x,y))
    return numerator/denominator
"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)

"""
8.0. tanimoto_distance K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= tanimoto_distance)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("tanimoto_distance K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))



txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016PMI.1M.txt', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016PMI.2F.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016PMI.1M.txt 100
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016PMI.2F.txt 100
length of features used = 200
len(all_training_text) 383
len(X_train)= 383 len(y_train)= 383
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:259: RuntimeWarning: invalid value encountered in double_scalars


tanimoto_distance K-Nearest Neighbours
0.5384615384615384


In [48]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "//Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016CHI.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016IG.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016GSS.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016OR.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016PMI.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016PMI.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016RF.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016RF.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016WLLR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016WLLR.2F.txt"]

print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        word_list2 =  ([r.strip() for r in f])[0:100]
        word_list1 =  ([(r.split(":"))[1] for r in word_list2])
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
#print("selector = SelectKBest(score_func=f_classif, k=200)")
#selector = SelectKBest(score_func=f_classif, k=200)
#fit = selector.fit(X_train,y_train)
#train_selected_features = fit.transform(X_train)
#test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
#X_train =train_selected_features 
#X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

def tanimoto_distance(x,y):
    """Calculates the tanimoto (normalised manhattan) distance between the vectors x and y
            Keyword arguments:
                x,y -- the vectors between which the distance is to be calculated
    """
    try:
        iter(x)
    except TypeError:
        logging.warning( 'Argument x is not iterable. None is returned')
        return None
    try:
        iter(y)
    except TypeError:
        logging.warning( 'Argument y is not iterable. None is returned')
        return None
    numerator = sum(abs(a - b) for a, b in zip(x, y))
    denominator = sum(max(a,b) for a,b in zip(x,y))
    return numerator/denominator
"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)

"""
8.0. tanimoto_distance K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= tanimoto_distance)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("tanimoto_distance K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))




txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016RF.1M.txt', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016RF.2F.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016RF.1M.txt 100
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016RF.2F.txt 100
length of features used = 200
len(all_training_text) 383
len(X_train)= 383 len(y_train)= 383
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:259: RuntimeWarning: invalid value encountered in double_scalars


tanimoto_distance K-Nearest Neighbours
0.5769230769230769


In [49]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "//Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016CHI.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016IG.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016GSS.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016OR.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016PMI.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016PMI.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016RF.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016RF.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016WLLR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016WLLR.2F.txt"]

print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        word_list2 =  ([r.strip() for r in f])[0:100]
        word_list1 =  ([(r.split(":"))[1] for r in word_list2])
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
#print("selector = SelectKBest(score_func=f_classif, k=200)")
#selector = SelectKBest(score_func=f_classif, k=200)
#fit = selector.fit(X_train,y_train)
#train_selected_features = fit.transform(X_train)
#test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
#X_train =train_selected_features 
#X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

def tanimoto_distance(x,y):
    """Calculates the tanimoto (normalised manhattan) distance between the vectors x and y
            Keyword arguments:
                x,y -- the vectors between which the distance is to be calculated
    """
    try:
        iter(x)
    except TypeError:
        logging.warning( 'Argument x is not iterable. None is returned')
        return None
    try:
        iter(y)
    except TypeError:
        logging.warning( 'Argument y is not iterable. None is returned')
        return None
    numerator = sum(abs(a - b) for a, b in zip(x, y))
    denominator = sum(max(a,b) for a,b in zip(x,y))
    return numerator/denominator
"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)

"""
8.0. tanimoto_distance K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= tanimoto_distance)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("tanimoto_distance K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))




txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016WLLR.1M.txt', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016WLLR.2F.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016WLLR.1M.txt 100
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2016TermSelection/outputNewVersion2016WLLR.2F.txt 100
length of features used = 200
len(all_training_text) 383
len(X_train)= 383 len(y_train)= 383
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
tanimoto_distance K-Nearest Neighbours
0.5128205128205128


In [2]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2...f_classif...mutual_info_classif.......................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=chi2, k=200)")
selector = SelectKBest(score_func=chi2, k=200)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

def tanimoto_distance(x,y):
    """Calculates the tanimoto (normalised manhattan) distance between the vectors x and y
            Keyword arguments:
                x,y -- the vectors between which the distance is to be calculated
    """
    try:
        iter(x)
    except TypeError:
        logging.warning( 'Argument x is not iterable. None is returned')
        return None
    try:
        iter(y)
    except TypeError:
        logging.warning( 'Argument y is not iterable. None is returned')
        return None
    numerator = sum(abs(a - b) for a, b in zip(x, y))
    denominator = sum(max(a,b) for a,b in zip(x,y))
    return numerator/denominator
"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)

"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))



txt_files of features used ['/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv 6328
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv 8793
length of features used = 15121
len(all_training_text) 383
len(X_train)= 383 len(y_train)= 383
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
selector = SelectKBest(score_func=chi2, k=200)
5
braycurtis K-Nearest Neighbours
[('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('F', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M')

In [3]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2...f_classif...mutual_info_classif.......................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=f_classif, k=200)")
selector = SelectKBest(score_func=f_classif, k=200)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

def tanimoto_distance(x,y):
    """Calculates the tanimoto (normalised manhattan) distance between the vectors x and y
            Keyword arguments:
                x,y -- the vectors between which the distance is to be calculated
    """
    try:
        iter(x)
    except TypeError:
        logging.warning( 'Argument x is not iterable. None is returned')
        return None
    try:
        iter(y)
    except TypeError:
        logging.warning( 'Argument y is not iterable. None is returned')
        return None
    numerator = sum(abs(a - b) for a, b in zip(x, y))
    denominator = sum(max(a,b) for a,b in zip(x,y))
    return numerator/denominator
"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)

"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))




txt_files of features used ['/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv 6328
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv 8793
length of features used = 15121
len(all_training_text) 383
len(X_train)= 383 len(y_train)= 383
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
selector = SelectKBest(score_func=f_classif, k=200)
5
braycurtis K-Nearest Neighbours
[('F', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('F', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('F', 'M'), ('M', 'M'), ('F', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('F', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('F', 'M'), ('M', 'M'), ('M', 'M'), ('M',

In [4]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2...f_classif...mutual_info_classif.......................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=mutual_info_classif, k=200)")
selector = SelectKBest(score_func=mutual_info_classif, k=200)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

def tanimoto_distance(x,y):
    """Calculates the tanimoto (normalised manhattan) distance between the vectors x and y
            Keyword arguments:
                x,y -- the vectors between which the distance is to be calculated
    """
    try:
        iter(x)
    except TypeError:
        logging.warning( 'Argument x is not iterable. None is returned')
        return None
    try:
        iter(y)
    except TypeError:
        logging.warning( 'Argument y is not iterable. None is returned')
        return None
    numerator = sum(abs(a - b) for a, b in zip(x, y))
    denominator = sum(max(a,b) for a,b in zip(x,y))
    return numerator/denominator
"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)

"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))





txt_files of features used ['/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv 6328
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv 8793
length of features used = 15121
len(all_training_text) 383
len(X_train)= 383 len(y_train)= 383
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
selector = SelectKBest(score_func=mutual_info_classif, k=200)
5
braycurtis K-Nearest Neighbours
[('F', 'M'), ('F', 'M'), ('F', 'M'), ('M', 'M'), ('M', 'M'), ('F', 'M'), ('F', 'M'), ('M', 'M'), ('F', 'M'), ('M', 'M'), ('F', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('F', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('F', 'M'), ('M', 'M'), ('M', '

In [3]:
#changing the classifiers..........


import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
#print("selector = SelectKBest(score_func=chi2, k=200)")
#selector = SelectKBest(score_func=chi2, k=200)
#fit = selector.fit(X_train,y_train)
#train_selected_features = fit.transform(X_train)
#test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
#X_train =train_selected_features 
#X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")

k=13

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("****************************************************************")


from sklearn import model_selection, naive_bayes, svm
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC



print("Naive****************************************************************")
Naive = naive_bayes.MultinomialNB()
Naive.fit(X_train, y_train)
y_pred = Naive.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))


print("SVM****************************************************************")
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train, y_train)
y_pred = SVM.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))


print("RandomForest_100****************************************************************")
clf=RandomForestClassifier(n_estimators=100)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

print("RandomForest_200****************************************************************")
clf=RandomForestClassifier(n_estimators=200)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))




txt_files of features used ['/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv 6328
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv 8793
length of features used = 15121
len(all_training_text) 383
len(X_train)= 383 len(y_train)= 383
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
manhattan K-Nearest Neighbours
0.5128205128205128
************************************ ****************************
13
manhattan K-Nearest Neighbours
0.48717948717948717
****************************************************************
Naive****************************************************************
0.6025641025641025
SVM***************************************************************

/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


0.5
RandomForest_100****************************************************************
0.5384615384615384
RandomForest_200****************************************************************
0.5384615384615384


In [4]:
#changing the classifiers..........


import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_vocubulary.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_ vocubulary_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
#print("selector = SelectKBest(score_func=chi2, k=200)")
#selector = SelectKBest(score_func=chi2, k=200)
#fit = selector.fit(X_train,y_train)
#train_selected_features = fit.transform(X_train)
#test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
#X_train =train_selected_features 
#X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")

k=13

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("****************************************************************")


from sklearn import model_selection, naive_bayes, svm
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC



print("Naive****************************************************************")
Naive = naive_bayes.MultinomialNB()
Naive.fit(X_train, y_train)
y_pred = Naive.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))


print("SVM****************************************************************")
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train, y_train)
y_pred = SVM.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))


print("RandomForest_100****************************************************************")
clf=RandomForestClassifier(n_estimators=100)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

print("RandomForest_200****************************************************************")
clf=RandomForestClassifier(n_estimators=200)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))





txt_files of features used ['/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_vocubulary.1.csv', '/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_ vocubulary_set.1.csv']
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_vocubulary.1.csv 115184
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_ vocubulary_set.1.csv 139886
length of features used = 255070
len(all_training_text) 383
len(X_train)= 383 len(y_train)= 383
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
manhattan K-Nearest Neighbours
0.5256410256410257
************************************ ****************************
13
manhattan K-Nearest Neighbours
0.5
****************************************************************
Naive****************************************************************
0.5
SVM****************************************************************
0.5
RandomFore

In [6]:
#changing the classifiers..........


import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_vocubulary.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_ vocubulary_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
#print("selector = SelectKBest(score_func=chi2, k=200)")
#selector = SelectKBest(score_func=chi2, k=200)
#fit = selector.fit(X_train,y_train)
#train_selected_features = fit.transform(X_train)
#test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
#X_train =train_selected_features 
#X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""


k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
print("manhattan_k=5************************************ ****************************")
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
manhattan_k5voc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")

k=13

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
print("manhattan_k=13************************************ ****************************")
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
manhattan_k13voc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))
print("****************************************************************")


from sklearn import model_selection, naive_bayes, svm
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier


print("MLPClassifier****************************************************************")
clf = MLPClassifier()
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
MLPClassifiervoc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("LogisticRegression****************************************************************")
clf = LogisticRegression()
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
LogisticRegressionvoc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("SGDClassifier****************************************************************")
clf = SGDClassifier()
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
SGDClassifiervoc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("DecisionTreeClassifier****************************************************************")
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()
# Train Decision Tree Classifer
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
DecisionTreeClassifiervoc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("Naive****************************************************************")
Naive = naive_bayes.MultinomialNB()
Naive.fit(X_train, y_train)
y_pred = Naive.predict(X_test)
Naivevoc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("SVM****************************************************************")
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train, y_train)
y_pred = SVM.predict(X_test)
SVMvoc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("RandomForest_100****************************************************************")
clf=RandomForestClassifier(n_estimators=100)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
RandomForest_100voc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))

print("RandomForest_200****************************************************************")
clf=RandomForestClassifier(n_estimators=200)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
RandomForest_200voc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))

from xgboost.sklearn import XGBClassifier 
print("XGBClassifier****************************************************************")
clf=XGBClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
XGBClassifiervoc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))



print("BaggingClassifier****************************************************************")
from sklearn.ensemble.bagging import BaggingClassifier
clf=BaggingClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
BaggingClassifiervoc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("AdaBoostClassifier****************************************************************")
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
clf=AdaBoostClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
AdaBoostClassifiervoc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))









txt_files of features used ['/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_vocubulary.1.csv', '/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_ vocubulary_set.1.csv']
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_vocubulary.1.csv 115184
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_ vocubulary_set.1.csv 139886
length of features used = 255070
len(all_training_text) 383
len(X_train)= 383 len(y_train)= 383
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
manhattan_k=5************************************ ****************************
manhattan K-Nearest Neighbours
0.5256410256410257
************************************ ****************************
13
manhattan_k=13************************************ ****************************
manhattan K-Nearest Neighbours
0.5
**************************************************************

/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


0.6153846153846154
LogisticRegression****************************************************************
0.5128205128205128
SGDClassifier****************************************************************


/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.5
DecisionTreeClassifier****************************************************************
0.5641025641025641
Naive****************************************************************
0.5
SVM****************************************************************
0.5
RandomForest_100****************************************************************
0.5641025641025641
RandomForest_200****************************************************************
0.5512820512820513
XGBClassifier****************************************************************


/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.6025641025641025
BaggingClassifier****************************************************************
0.5897435897435898
AdaBoostClassifier****************************************************************
0.6282051282051282


In [11]:

print("The sign test...****************************************************************")
print("manhattan_k=5************************************ ****************************")
x= AdaBoostClassifiervoc 
y= manhattan_k5voc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)
        
print("manhattan_k=13************************************ ****************************")
x= AdaBoostClassifiervoc 
y= manhattan_k13voc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)

print("MLPClassifier****************************************************************")
x= AdaBoostClassifiervoc 
y= MLPClassifiervoc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("LogisticRegression****************************************************************")
x= AdaBoostClassifiervoc 
y= LogisticRegressionvoc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("SGDClassifier****************************************************************")
x= AdaBoostClassifiervoc 
y= SGDClassifiervoc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("DecisionTreeClassifier****************************************************************")
x= AdaBoostClassifiervoc 
y= DecisionTreeClassifiervoc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("Naive****************************************************************")
x= AdaBoostClassifiervoc 
y= Naivevoc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("SVM****************************************************************")
x= AdaBoostClassifiervoc 
y= SVMvoc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("RandomForest_100****************************************************************")
x= AdaBoostClassifiervoc 
y= RandomForest_100voc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("RandomForest_200****************************************************************")
x= AdaBoostClassifiervoc 
y= RandomForest_200voc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("XGBClassifier****************************************************************")
x= AdaBoostClassifiervoc 
y= XGBClassifiervoc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("BaggingClassifier****************************************************************")
x= AdaBoostClassifiervoc 
y= BaggingClassifiervoc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("AdaBoostClassifier****************************************************************")
x= AdaBoostClassifiervoc 
y= AdaBoostClassifiervoc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)





The sign test...****************************************************************
manhattan_k=5************************************ ****************************
positive= 26 negative = 18
manhattan_k=13************************************ ****************************
positive= 27 negative = 17
MLPClassifier****************************************************************
positive= 17 negative = 16
LogisticRegression****************************************************************
positive= 27 negative = 18
SGDClassifier****************************************************************
positive= 28 negative = 18
DecisionTreeClassifier****************************************************************
positive= 20 negative = 15
Naive****************************************************************
positive= 26 negative = 16
SVM****************************************************************
positive= 21 negative = 11
RandomForest_100****************************************************************

In [7]:
#changing the classifiers..........


import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
#print("selector = SelectKBest(score_func=chi2, k=200)")
#selector = SelectKBest(score_func=chi2, k=200)
#fit = selector.fit(X_train,y_train)
#train_selected_features = fit.transform(X_train)
#test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
#X_train =train_selected_features 
#X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
print("manhattan_k=5************************************ ****************************")
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
manhattan_k5fs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")

k=13

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
print("manhattan_k=13************************************ ****************************")
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
manhattan_k13fs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))
print("****************************************************************")


from sklearn import model_selection, naive_bayes, svm
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier


print("MLPClassifier****************************************************************")
clf = MLPClassifier()
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
MLPClassifierfs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("LogisticRegression****************************************************************")
clf = LogisticRegression()
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
LogisticRegressionfs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("SGDClassifier****************************************************************")
clf = SGDClassifier()
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
SGDClassifierfs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("DecisionTreeClassifier****************************************************************")
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()
# Train Decision Tree Classifer
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
DecisionTreeClassifierfs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("Naive****************************************************************")
Naive = naive_bayes.MultinomialNB()
Naive.fit(X_train, y_train)
y_pred = Naive.predict(X_test)
Naivefs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("SVM****************************************************************")
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train, y_train)
y_pred = SVM.predict(X_test)
SVMfs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("RandomForest_100****************************************************************")
clf=RandomForestClassifier(n_estimators=100)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
RandomForest_100fs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))

print("RandomForest_200****************************************************************")
clf=RandomForestClassifier(n_estimators=200)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
RandomForest_200fs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))

from xgboost.sklearn import XGBClassifier 
print("XGBClassifier****************************************************************")
clf=XGBClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
XGBClassifierfs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))



print("BaggingClassifier****************************************************************")
from sklearn.ensemble.bagging import BaggingClassifier
clf=BaggingClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
BaggingClassifierfs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("AdaBoostClassifier****************************************************************")
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
clf=AdaBoostClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
AdaBoostClassifierfs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))




txt_files of features used ['/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv 6328
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv 8793
length of features used = 15121
len(all_training_text) 383
len(X_train)= 383 len(y_train)= 383
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
manhattan_k=5************************************ ****************************
manhattan K-Nearest Neighbours
0.5128205128205128
************************************ ****************************
13
manhattan_k=13************************************ ****************************
manhattan K-Nearest Neighbours
0.48717948717948717
*************************************************************

/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.5769230769230769
LogisticRegression****************************************************************
0.5128205128205128
SGDClassifier****************************************************************
0.5
DecisionTreeClassifier****************************************************************
0.6282051282051282
Naive****************************************************************
0.6025641025641025
SVM****************************************************************
0.5
RandomForest_100****************************************************************
0.5128205128205128
RandomForest_200****************************************************************
0.5384615384615384
XGBClassifier****************************************************************


/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.6410256410256411
BaggingClassifier****************************************************************
0.6538461538461539
AdaBoostClassifier****************************************************************
0.6282051282051282


In [10]:
print("The sign test...****************************************************************")
print("manhattan_k=5************************************ ****************************")
x= BaggingClassifierfs 
y= manhattan_k5fs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)
        
print("manhattan_k=13************************************ ****************************")
x= BaggingClassifierfs 
y= manhattan_k13fs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)

print("MLPClassifier****************************************************************")
x= BaggingClassifierfs 
y= MLPClassifierfs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("LogisticRegression****************************************************************")
x= BaggingClassifierfs 
y= LogisticRegressionfs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("SGDClassifier****************************************************************")
x= BaggingClassifierfs 
y= SGDClassifierfs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("DecisionTreeClassifier****************************************************************")
x= BaggingClassifierfs 
y= DecisionTreeClassifierfs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("Naive****************************************************************")
x= BaggingClassifierfs 
y= Naivefs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("SVM****************************************************************")
x= BaggingClassifierfs 
y= SVMfs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("RandomForest_100****************************************************************")
x= BaggingClassifierfs 
y= RandomForest_100fs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("RandomForest_200****************************************************************")
x= BaggingClassifierfs 
y= RandomForest_200fs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("XGBClassifier****************************************************************")
x= BaggingClassifierfs 
y= XGBClassifierfs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("BaggingClassifier****************************************************************")
x= BaggingClassifierfs 
y= BaggingClassifierfs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("AdaBoostClassifier****************************************************************")
x= BaggingClassifierfs 
y= AdaBoostClassifierfs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)







The sign test...****************************************************************
manhattan_k=5************************************ ****************************
positive= 23 negative = 12
manhattan_k=13************************************ ****************************
positive= 25 negative = 12
MLPClassifier****************************************************************
positive= 18 negative = 12
LogisticRegression****************************************************************
positive= 23 negative = 12
SGDClassifier****************************************************************
positive= 27 negative = 15
DecisionTreeClassifier****************************************************************
positive= 17 negative = 15
Naive****************************************************************
positive= 18 negative = 14
SVM****************************************************************
positive= 27 negative = 15
RandomForest_100****************************************************************

In [3]:
#..................................................................................

data_tags = ["type","gender","tweet"]
train_data_list = []
i=0
for f in y_train:
   
    train_data_list.append((f, y_train[i],all_training_text[i]))
    i += 1
# We have training data available as dictionary filename, category, data
train_data = pd.DataFrame.from_records(train_data_list, columns=data_tags)




#..................................................................................

test_data_list = []
ii=0
for f in y_test:
   
    test_data_list.append((f, y_test[ii], all_test_text[ii]))
    ii += 1
# We have training data available as dictionary filename, category, data
test_data = pd.DataFrame.from_records(test_data_list, columns=data_tags)

#..................................................................................


import pandas as pd
import numpy as np
import pickle
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
import sklearn.datasets as skds
from pathlib import Path

Using TensorFlow backend.
/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
  return f(*args, **kwds)


In [4]:

train_posts = train_data['tweet']
train_tags = train_data['type']

test_posts = test_data['tweet']
test_tags = test_data['type']




In [5]:
# :Tokenize and Prepare Vocabulary
num_labels = 2
#vocab_size = 15000#len(word_list)#most common number of words will be then kept for use in the vector
vocab_size = 1000#by changing the vocubulary size... acc count_type =  0.932258064516129#acc count_gender = 0.8298387096774194
#vocab_size = 100#acc count_type =0.9274193548387096 acc count_gender = 0.792741935483871
batch_size = 100
 
# define Tokenizer with Vocab Size
tokenizer = Tokenizer(num_words=vocab_size)#IF I USE OTHER TOKENIZER...
tokenizer.fit_on_texts(train_posts)
#print(tokenizer.word_index)
#print((tokenizer.word_counts))#provides a dictionary of the words and the count......................
#sorted_x = sorted((tokenizer.word_counts).items(), key=operator.itemgetter(1),reverse=True)
#print(sorted_x)
#print((tokenizer.document_count))#number of documents..............
#print((tokenizer.word_docs))#provides a dictionary of the words and the number of documents they appear in......................
print('Found %d unique words.' % len(tokenizer.word_index))#shows total vocubulary of the text dataset
x_train = tokenizer.texts_to_matrix(train_posts, mode='tfidf')#WHAT HAPPENS WHEN I GIVE IT THE HAND CRAFTED TOKENS??
x_test = tokenizer.texts_to_matrix(test_posts, mode='tfidf')

##--------what about applying knn at this point-------------------------------
##--------what about changing the vocubulary size-------------------------------works well with reduced sized
##----------One popular method for hyperparameter optimization is grid search.-----------
##-----determine the best set of parameters with the highest accuracy..........this is for the keras model------


    
    
encoder = LabelBinarizer()


encoder.fit(train_tags)

y_train = encoder.transform(train_tags)#same as the y train generated with my model
y_train = np.hstack((y_train, 1 - y_train))#used for two label cases........

y_test = encoder.transform(test_tags)
y_test = np.hstack((y_test, 1 - y_test))

"""
fit_on_texts Updates internal vocabulary based on a list of texts. This method creates the vocabulary index based on word frequency. So if you give it something like, "The cat sat on the mat." It will create a dictionary s.t. word_index["the"] = 0; word_index["cat"] = 1 it is word -> index dictionary so every word gets a unique integer value. So lower integer means more frequent word (often the first few are punctuation because they appear a lot).
texts_to_sequences Transforms each text in texts to a sequence of integers. So it basically takes each word in the text and replaces it with its corresponding integer value from the word_index dictionary. Nothing more, nothing less, certainly no magic involved.
CLEAN TEXT TO THE DESIRED LEVEL AND USE KERAS INBUILF TFIDF TO GENERATE A MATRIX....
"""

Found 221453 unique words.


'\nfit_on_texts Updates internal vocabulary based on a list of texts. This method creates the vocabulary index based on word frequency. So if you give it something like, "The cat sat on the mat." It will create a dictionary s.t. word_index["the"] = 0; word_index["cat"] = 1 it is word -> index dictionary so every word gets a unique integer value. So lower integer means more frequent word (often the first few are punctuation because they appear a lot).\ntexts_to_sequences Transforms each text in texts to a sequence of integers. So it basically takes each word in the text and replaces it with its corresponding integer value from the word_index dictionary. Nothing more, nothing less, certainly no magic involved.\nCLEAN TEXT TO THE DESIRED LEVEL AND USE KERAS INBUILF TFIDF TO GENERATE A MATRIX....\n'

In [6]:
from keras.models import *
from keras.layers import *

model = Sequential()
model.add(Dense(512, input_shape=(vocab_size,)))#512 neurons in the first hidden layer
model.add(Activation('relu'))
#model.add(Dropout(0.3))
model.add(Dense(512))
model.add(Activation('relu'))
#model.add(Dropout(0.3))
model.add(Dense(num_labels))
model.add(Activation('softmax'))
model.summary()
 
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=30,
                    verbose=1,
                    validation_split=0.1)



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               512512    
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
activation_2 (Activation)    (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 1026      
_________________________________________________________________
activation_3 (Activation)    (None, 2)                 0         
Total params: 776,194
Trainable params: 776,194
Non-trainable params: 0
_________________________________________________________________
Trai

In [7]:
#Evaluate model.............................
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
 
print('Test accuracy:', score[1])

78/78 [==============================] - 0s 33us/step
Test accuracy: 0.7435897588729858


In [ ]:
#changing the classifiers..........


import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
#X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
#print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
#X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
#print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)



#..................................................................................

data_tags = ["type","gender","tweet"]
train_data_list = []
i=0
for f in y_train:
   
    train_data_list.append((f, y_train[i],all_training_text[i]))
    i += 1
# We have training data available as dictionary filename, category, data
train_data = pd.DataFrame.from_records(train_data_list, columns=data_tags)




#..................................................................................

test_data_list = []
ii=0
for f in y_test:
   
    test_data_list.append((f, y_test[ii], all_test_text[ii]))
    ii += 1
# We have training data available as dictionary filename, category, data
test_data = pd.DataFrame.from_records(test_data_list, columns=data_tags)

#..................................................................................


import pandas as pd
import numpy as np
import pickle
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
import sklearn.datasets as skds
from pathlib import Path

# :Tokenize and Prepare Vocabulary

train_posts = train_data['tweet']
train_tags = train_data['type']

test_posts = test_data['tweet']
test_tags = test_data['type']


num_labels = 2
vocab_size = 1000#len(word_list)#most common number of words will be then kept for use in the vector
#vocab_size = 1000#by changing the vocubulary size... acc count_type =  0.932258064516129#acc count_gender = 0.8298387096774194
#vocab_size = 100#acc count_type =0.9274193548387096 acc count_gender = 0.792741935483871
batch_size = 100
 
# define Tokenizer with Vocab Size
tokenizer = Tokenizer(num_words=vocab_size)#IF I USE OTHER TOKENIZER...
tokenizer.fit_on_texts(train_posts)
#print(tokenizer.word_index)
#print((tokenizer.word_counts))#provides a dictionary of the words and the count......................
#sorted_x = sorted((tokenizer.word_counts).items(), key=operator.itemgetter(1),reverse=True)
#print(sorted_x)
#print((tokenizer.document_count))#number of documents..............
#print((tokenizer.word_docs))#provides a dictionary of the words and the number of documents they appear in......................
print('Found %d unique words.' % len(tokenizer.word_index))#shows total vocubulary of the text dataset
x_train = tokenizer.texts_to_matrix(train_posts, mode='tfidf')#WHAT HAPPENS WHEN I GIVE IT THE HAND CRAFTED TOKENS??
x_test = tokenizer.texts_to_matrix(test_posts, mode='tfidf')

##--------what about applying knn at this point-------------------------------
##--------what about changing the vocubulary size-------------------------------works well with reduced sized
##----------One popular method for hyperparameter optimization is grid search.-----------
##-----determine the best set of parameters with the highest accuracy..........this is for the keras model------


      
encoder = LabelBinarizer()


encoder.fit(train_tags)

y_train = encoder.transform(train_tags)#same as the y train generated with my model
y_train = np.hstack((y_train, 1 - y_train))#used for two label cases........

y_test = encoder.transform(test_tags)
y_test = np.hstack((y_test, 1 - y_test))


"""
fit_on_texts Updates internal vocabulary based on a list of texts. This method creates the vocabulary index based on word frequency. So if you give it something like, "The cat sat on the mat." It will create a dictionary s.t. word_index["the"] = 0; word_index["cat"] = 1 it is word -> index dictionary so every word gets a unique integer value. So lower integer means more frequent word (often the first few are punctuation because they appear a lot).
texts_to_sequences Transforms each text in texts to a sequence of integers. So it basically takes each word in the text and replaces it with its corresponding integer value from the word_index dictionary. Nothing more, nothing less, certainly no magic involved.
CLEAN TEXT TO THE DESIRED LEVEL AND USE KERAS INBUILF TFIDF TO GENERATE A MATRIX....
"""
from sklearn import model_selection, naive_bayes, svm
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
#8. manhattan K-Nearest Neighbours..............................................................................

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
print("manhattan_k=5************************************ ****************************")
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(x_train, train_tags)
y_pred = knn.predict(x_test)
manhattan_k5tfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(test_tags, y_pred))
print("************************************ ****************************")

k=13

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
print("manhattan_k=13************************************ ****************************")
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(x_train, train_tags)
y_pred = knn.predict(x_test)
manhattan_k13tfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(test_tags, y_pred))
print("****************************************************************")


from sklearn import model_selection, naive_bayes, svm
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier


print("MLPClassifier****************************************************************")
clf = MLPClassifier()
clf = clf.fit(x_train, train_tags)
#Predict the response for test dataset
y_pred = clf.predict(x_test)
MLPClassifiertfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))


print("LogisticRegression****************************************************************")
clf = LogisticRegression()
clf = clf.fit(x_train, train_tags)
#Predict the response for test dataset
y_pred = clf.predict(x_test)
LogisticRegressiontfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))


print("SGDClassifier****************************************************************")
clf = SGDClassifier()
clf = clf.fit(x_train, train_tags)
#Predict the response for test dataset
y_pred = clf.predict(x_test)
SGDClassifiertfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))


print("DecisionTreeClassifier****************************************************************")
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()
# Train Decision Tree Classifer
clf = clf.fit(x_train, train_tags)
#Predict the response for test dataset
y_pred = clf.predict(x_test)
DecisionTreeClassifiertfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))


print("Naive****************************************************************")
Naive = naive_bayes.MultinomialNB()
Naive.fit(x_train, train_tags)
y_pred = Naive.predict(x_test)
Naivetfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))


print("SVM****************************************************************")
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(x_train, train_tags)
y_pred = SVM.predict(x_test)
SVMtfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))


print("RandomForest_100****************************************************************")
clf=RandomForestClassifier(n_estimators=100)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train,train_tags)
y_pred=clf.predict(x_test)
RandomForest_100tfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))

print("RandomForest_200****************************************************************")
clf=RandomForestClassifier(n_estimators=200)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train,train_tags)
y_pred=clf.predict(x_test)
RandomForest_200tfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))

from xgboost.sklearn import XGBClassifier 
print("XGBClassifier****************************************************************")
clf=XGBClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train, train_tags)
y_pred=clf.predict(x_test)
XGBClassifiertfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))



print("BaggingClassifier****************************************************************")
from sklearn.ensemble.bagging import BaggingClassifier
clf=BaggingClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train, train_tags)
y_pred=clf.predict(x_test)
BaggingClassifiertfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))


print("AdaBoostClassifier****************************************************************")
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
clf=AdaBoostClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train, train_tags)
y_pred=clf.predict(x_test)
AdaBoostClassifiertfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))





from keras.models import *
from keras.layers import *

model = Sequential()
model.add(Dense(512, input_shape=(vocab_size,)))#512 neurons in the first hidden layer
model.add(Activation('relu'))
#model.add(Dropout(0.3))
model.add(Dense(512))
model.add(Activation('relu'))
#model.add(Dropout(0.3))
model.add(Dense(num_labels))
model.add(Activation('softmax'))
model.summary()
 
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=30,
                    verbose=1,
                    validation_split=0.1)



#Evaluate model.............................
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
 
print('Test accuracy:', score[1])



In [ ]:
#changing the classifiers..........


import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
#X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
#print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
#X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
#print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)



#..................................................................................

data_tags = ["type","gender","tweet"]
train_data_list = []
i=0
for f in y_train:
   
    train_data_list.append((f, y_train[i],all_training_text[i]))
    i += 1
# We have training data available as dictionary filename, category, data
train_data = pd.DataFrame.from_records(train_data_list, columns=data_tags)




#..................................................................................

test_data_list = []
ii=0
for f in y_test:
   
    test_data_list.append((f, y_test[ii], all_test_text[ii]))
    ii += 1
# We have training data available as dictionary filename, category, data
test_data = pd.DataFrame.from_records(test_data_list, columns=data_tags)

#..................................................................................


import pandas as pd
import numpy as np
import pickle
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
import sklearn.datasets as skds
from pathlib import Path

# :Tokenize and Prepare Vocabulary

train_posts = train_data['tweet']
train_tags = train_data['type']

test_posts = test_data['tweet']
test_tags = test_data['type']


num_labels = 2
vocab_size = 1000#len(word_list)#most common number of words will be then kept for use in the vector
#vocab_size = 1000#by changing the vocubulary size... acc count_type =  0.932258064516129#acc count_gender = 0.8298387096774194
#vocab_size = 100#acc count_type =0.9274193548387096 acc count_gender = 0.792741935483871
batch_size = 100
 
# define Tokenizer with Vocab Size
tokenizer = Tokenizer(num_words=vocab_size)#IF I USE OTHER TOKENIZER...
tokenizer.fit_on_texts(train_posts)
#print(tokenizer.word_index)
#print((tokenizer.word_counts))#provides a dictionary of the words and the count......................
#sorted_x = sorted((tokenizer.word_counts).items(), key=operator.itemgetter(1),reverse=True)
#print(sorted_x)
#print((tokenizer.document_count))#number of documents..............
#print((tokenizer.word_docs))#provides a dictionary of the words and the number of documents they appear in......................
print('Found %d unique words.' % len(tokenizer.word_index))#shows total vocubulary of the text dataset
x_train = tokenizer.texts_to_matrix(train_posts, mode='tfidf')#WHAT HAPPENS WHEN I GIVE IT THE HAND CRAFTED TOKENS??
x_test = tokenizer.texts_to_matrix(test_posts, mode='tfidf')

##--------what about applying knn at this point-------------------------------
##--------what about changing the vocubulary size-------------------------------works well with reduced sized
##----------One popular method for hyperparameter optimization is grid search.-----------
##-----determine the best set of parameters with the highest accuracy..........this is for the keras model------


      
encoder = LabelBinarizer()


encoder.fit(train_tags)

y_train = encoder.transform(train_tags)#same as the y train generated with my model
y_train = np.hstack((y_train, 1 - y_train))#used for two label cases........

y_test = encoder.transform(test_tags)
y_test = np.hstack((y_test, 1 - y_test))


"""
fit_on_texts Updates internal vocabulary based on a list of texts. This method creates the vocabulary index based on word frequency. So if you give it something like, "The cat sat on the mat." It will create a dictionary s.t. word_index["the"] = 0; word_index["cat"] = 1 it is word -> index dictionary so every word gets a unique integer value. So lower integer means more frequent word (often the first few are punctuation because they appear a lot).
texts_to_sequences Transforms each text in texts to a sequence of integers. So it basically takes each word in the text and replaces it with its corresponding integer value from the word_index dictionary. Nothing more, nothing less, certainly no magic involved.
CLEAN TEXT TO THE DESIRED LEVEL AND USE KERAS INBUILF TFIDF TO GENERATE A MATRIX....
"""
from sklearn import model_selection, naive_bayes, svm
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
#8. manhattan K-Nearest Neighbours..............................................................................

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
print("manhattan_k=5************************************ ****************************")
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(x_train, train_tags)
y_pred = knn.predict(x_test)
manhattan_k5tfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(test_tags, y_pred))
print("************************************ ****************************")

k=13

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
print("manhattan_k=13************************************ ****************************")
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(x_train, train_tags)
y_pred = knn.predict(x_test)
manhattan_k13tfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(test_tags, y_pred))
print("****************************************************************")


from sklearn import model_selection, naive_bayes, svm
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier


print("MLPClassifier****************************************************************")
clf = MLPClassifier()
clf = clf.fit(x_train, train_tags)
#Predict the response for test dataset
y_pred = clf.predict(x_test)
MLPClassifiertfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))


print("LogisticRegression****************************************************************")
clf = LogisticRegression()
clf = clf.fit(x_train, train_tags)
#Predict the response for test dataset
y_pred = clf.predict(x_test)
LogisticRegressiontfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))


print("SGDClassifier****************************************************************")
clf = SGDClassifier()
clf = clf.fit(x_train, train_tags)
#Predict the response for test dataset
y_pred = clf.predict(x_test)
SGDClassifiertfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))


print("DecisionTreeClassifier****************************************************************")
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()
# Train Decision Tree Classifer
clf = clf.fit(x_train, train_tags)
#Predict the response for test dataset
y_pred = clf.predict(x_test)
DecisionTreeClassifiertfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))


print("Naive****************************************************************")
Naive = naive_bayes.MultinomialNB()
Naive.fit(x_train, train_tags)
y_pred = Naive.predict(x_test)
Naivetfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))


print("SVM****************************************************************")
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(x_train, train_tags)
y_pred = SVM.predict(x_test)
SVMtfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))


print("RandomForest_100****************************************************************")
clf=RandomForestClassifier(n_estimators=100)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train,train_tags)
y_pred=clf.predict(x_test)
RandomForest_100tfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))

print("RandomForest_200****************************************************************")
clf=RandomForestClassifier(n_estimators=200)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train,train_tags)
y_pred=clf.predict(x_test)
RandomForest_200tfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))

from xgboost.sklearn import XGBClassifier 
print("XGBClassifier****************************************************************")
clf=XGBClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train, train_tags)
y_pred=clf.predict(x_test)
XGBClassifiertfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))



print("BaggingClassifier****************************************************************")
from sklearn.ensemble.bagging import BaggingClassifier
clf=BaggingClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train, train_tags)
y_pred=clf.predict(x_test)
BaggingClassifiertfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))


print("AdaBoostClassifier****************************************************************")
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
clf=AdaBoostClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train, train_tags)
y_pred=clf.predict(x_test)
AdaBoostClassifiertfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))





from keras.models import *
from keras.layers import *

model = Sequential()
model.add(Dense(512, input_shape=(vocab_size,)))#512 neurons in the first hidden layer
model.add(Activation('relu'))
#model.add(Dropout(0.3))
model.add(Dense(512))
model.add(Activation('relu'))
#model.add(Dropout(0.3))
model.add(Dense(num_labels))
model.add(Activation('softmax'))
model.summary()
 
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=30,
                    verbose=1,
                    validation_split=0.1)



#Evaluate model.............................
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
 
print('Test accuracy:', score[1])




In [8]:
print("The sign test...****************************************************************")
print("manhattan_k=5************************************ ****************************")
x= SGDClassifiertfidf 
y= manhattan_k5tfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)
        
print("manhattan_k=13************************************ ****************************")
x= SGDClassifiertfidf 
y= manhattan_k13tfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)

print("MLPClassifier****************************************************************")
x= SGDClassifiertfidf 
y= MLPClassifiertfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("LogisticRegression****************************************************************")
x= SGDClassifiertfidf 
y= LogisticRegressiontfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("SGDClassifier****************************************************************")
x= SGDClassifiertfidf 
y= SGDClassifiertfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("DecisionTreeClassifier****************************************************************")
x= SGDClassifiertfidf 
y= DecisionTreeClassifiertfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("Naive****************************************************************")
x= SGDClassifiertfidf 
y= Naivetfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("SVM****************************************************************")
x= SGDClassifiertfidf 
y= SVMtfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("RandomForest_100****************************************************************")
x= SGDClassifiertfidf 
y= RandomForest_100tfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("RandomForest_200****************************************************************")
x= SGDClassifiertfidf
y= RandomForest_200tfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("XGBClassifier****************************************************************")
x= SGDClassifiertfidf 
y= XGBClassifiertfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("BaggingClassifier****************************************************************")
x= SGDClassifiertfidf 
y= BaggingClassifiertfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("AdaBoostClassifier****************************************************************")
x= SGDClassifiertfidf
y= AdaBoostClassifiertfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




BaggingClassifier****************************************************************
0.5641025641025641
AdaBoostClassifier****************************************************************
0.5769230769230769


In [ ]:
#changing the classifiers..........


import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
#X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
#print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
#X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
#print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))




print("************************************ ****************************")


#..................................................................................

data_tags = ["type","gender","tweet"]
train_data_list = []
i=0
for f in y_train:
   
    train_data_list.append((f, y_train[i],all_training_text[i]))
    i += 1
# We have training data available as dictionary filename, category, data
train_data = pd.DataFrame.from_records(train_data_list, columns=data_tags)




#..................................................................................

test_data_list = []
ii=0
for f in y_test:
   
    test_data_list.append((f, y_test[ii], all_test_text[ii]))
    ii += 1
# We have training data available as dictionary filename, category, data
test_data = pd.DataFrame.from_records(test_data_list, columns=data_tags)

#..................................................................................


import pandas as pd
import numpy as np
import pickle
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
import sklearn.datasets as skds
from pathlib import Path
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers import Dense, Conv1D, Flatten
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers import Activation, Conv1D, Dense, Embedding, Flatten, Input, MaxPooling1D, Bidirectional


# :Tokenize and Prepare Vocabulary

train_posts = train_data['tweet']
train_tags = train_data['type']
train_tags=array([1 if x=="M" else 0 for x in train_tags])
#train_tags=train_tags.reshape(-1, 1)
#print("train_tags", print(train_tags.shape), (train_tags))

test_posts = test_data['tweet']
test_tags = test_data['type']
test_tags=[1 if x=="M" else 0 for x in test_tags]




# define train documents
docs = train_posts

# define train class labels
labels = train_tags

# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1

# integer encode the documents
encoded_docs = t.texts_to_sequences(docs)
#print(encoded_docs)

# pad documents to a max length of 4 words
max_length = 1000

padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

#print(padded_docs)



# define test documents
test_docs = test_posts

# define train class labels
test_labels = test_tags

# prepare tokenizer
test_t = Tokenizer()
test_t.fit_on_texts(test_docs)
test_vocab_size = len(test_t.word_index) + 1

# integer encode the documents
test_encoded_docs = test_t.texts_to_sequences(test_docs)
#print(encoded_docs)

# pad documents to a max length of 4 words

test_padded_docs = pad_sequences(test_encoded_docs, maxlen=max_length, padding='post')
#print(padded_docs)


# load the whole embedding into memory
embeddings_index = dict()
f = open('/Users/catherine/Downloads/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector


# create the model_5
from keras import layers
embedding_dim = 100
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=True))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

history = model.fit(X_train, y_train, epochs=10, verbose=False, validation_data=(test_padded_docs, test_labels), batch_size=10)
loss, accuracy = model.evaluate(padded_docs, labels, verbose=False)
print("odel_5 Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(test_padded_docs, test_labels, verbose=False)
print("odel_5 Testing Accuracy:  {:.4f}".format(accuracy))
plot_history(history)

        
# define model_1
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length, trainable= True))
model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())
# compile network
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(padded_docs, labels, epochs=10, verbose=2)
# evaluate
loss, acc = model.evaluate(test_padded_docs, test_labels, verbose=0)
print('model_1 Accuracy: %f' % (acc*100))
        
# define model_2
model = Sequential()
model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_length, trainable= True))
#model.add(Embedding(vocab_size, 100, input_length=max_length, trainable= False))
model.add(Conv1D(50, 5))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(padded_docs.shape)
model.fit(padded_docs, labels, epochs=10, batch_size=64)

loss, accuracy = model.evaluate(test_padded_docs, test_labels, verbose=0)
print('model_2 Accuracy: %f' % (accuracy*100))

# create the model_3
top_words = 5000
# create the model
embedding_vecor_length = 32
model = Sequential()
#model.add(Embedding(top_words, len(embedding_matrix), input_length=max_length))
model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_length, trainable= False))
#model.add(Dropout(0.2))
model.add(LSTM(100))
#model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(padded_docs, labels, epochs=10, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(test_padded_docs, test_labels, verbose=0)
print("model_3 Accuracy: %.2f%%" % (scores[1]*100))



# create the model_4
model = Sequential()
model.add(Embedding(len(embedding_matrix), 100, weights=[embedding_matrix], input_length=max_length, trainable=False))
model.add(Bidirectional(LSTM(128)))
#model.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(512, activation='relu'))
#model.add(Dropout(0.50))
model.add(Dense(1, activation='softmax'))
# Adam Optimiser
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(padded_docs, labels, epochs=10, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(test_padded_docs, test_labels, verbose=0)
print('model_4 Accuracy: %f' % (scores[1]*100))



# create the model_5
from keras import layers
embedding_dim = 100
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=True))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

history = model.fit(X_train, y_train, epochs=10, verbose=False, validation_data=(test_padded_docs, test_labels), batch_size=10)
loss, accuracy = model.evaluate(padded_docs, labels, verbose=False)
print("odel_5 Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(test_padded_docs, test_labels, verbose=False)
print("odel_5 Testing Accuracy:  {:.4f}".format(accuracy))
plot_history(history)